In [3]:
import boto3

In [5]:
import json

In [11]:
import os
from IPython.display import Audio
import boto3
import uuid
import time
import json
from jinja2 import Template

In [13]:
sound_clip = Audio(filename="dialog.mp3")
display(sound_clip)

In [15]:
cloud_storage = boto3.client('s3', region_name='us-east-1')

In [17]:
bucket_name = "genaiufds2"

In [19]:
file_name = 'dialog.mp3'

In [21]:
cloud_storage.upload_file(file_name, bucket_name, file_name)

In [23]:
speech_to_text = boto3.client('transcribe', region_name='us-east-1')

In [25]:
job_name = 'transcribe-job-' + str(uuid.uuid4())

In [27]:
job_name

'transcribe-job-8769e7ce-ebba-4184-b230-47d0458b9bfe'

In [29]:
response = speech_to_text.start_transcription_job(
    TranscriptionJobName=job_name,
    Media={'MediaFileUri': f's3://{bucket_name}/{file_name}'},
    MediaFormat='mp3',
    LanguageCode='en-US',
    OutputBucketName=bucket_name,
    Settings={
        'ShowSpeakerLabels': True,
        'MaxSpeakerLabels': 2
    }
)

In [31]:
while True:
    job_status = speech_to_text.get_transcription_job(TranscriptionJobName=job_name)
    if job_status['TranscriptionJob']['TranscriptionJobStatus'] in ['COMPLETED', 'FAILED']:
        break
    time.sleep(2)

print(job_status['TranscriptionJob']['TranscriptionJobStatus'])


COMPLETED


In [33]:
if job_status['TranscriptionJob']['TranscriptionJobStatus'] == 'COMPLETED':
    
    # Load the transcript from cloud storage.
    transcript_key = f"{job_name}.json"
    transcript_obj = cloud_storage.get_object(Bucket=bucket_name, Key=transcript_key)
    transcript_data = transcript_obj['Body'].read().decode('utf-8')
    transcript_json = json.loads(transcript_data)
    
    output_text = ""
    previous_speaker = None
    
    items = transcript_json['results']['items']
    
    for item in items:
        
        speaker_label = item.get('speaker_label', None)
        content = item['alternatives'][0]['content']
        
        # Start the line with the speaker label:
        if speaker_label is not None and speaker_label != previous_speaker:
            previous_speaker = speaker_label
            output_text += f"\n{previous_speaker}: "
            
        # Add the speech content:
        if item['type'] == 'punctuation':
            output_text = output_text.rstrip()
            
        output_text += f"{content} "
        
    # Save the transcript to a text file
    with open(f'{job_name}.txt', 'w') as f:
        f.write(output_text)


In [47]:
bedrock_runtime = boto3.client('bedrock-runtime', region_name='us-east-1')

In [45]:
with open(f'{job_name}.txt', "r") as file:
    transcript = file.read()
    #print(transcript)

In [49]:
%%writefile prompt_template.txt
I need to summarize a conversation. The transcript of the 
conversation is between the <data> XML like tags.

<data>
{{transcript}}
</data>

The summary must contain a one word sentiment analysis, and 
a list of issues, problems or causes of friction
during the conversation. The output must be provided in 
JSON format shown in the following example. 

Example output:
{
    "sentiment": <Overall sentiment>,
    "issues": [
        {
            "topic": <topic>,
            "summary": <issue_summary>,
        }
    ]
}

Write the JSON output.

Here is the JSON output:


Overwriting prompt_template.txt


In [51]:
with open('prompt_template.txt', "r") as file:
    template_string = file.read()

In [53]:
template_string

'I need to summarize a conversation. The transcript of the \nconversation is between the <data> XML like tags.\n\n<data>\n{{transcript}}\n</data>\n\nThe summary must contain a one word sentiment analysis, and \na list of issues, problems or causes of friction\nduring the conversation. The output must be provided in \nJSON format shown in the following example. \n\nExample output:\n{\n    "sentiment": <Overall sentiment>,\n    "issues": [\n        {\n            "topic": <topic>,\n            "summary": <issue_summary>,\n        }\n    ]\n}\n\nWrite the JSON output.\n\nHere is the JSON output:\n'

In [55]:
data = {
    'transcript' : transcript
}

In [57]:
template = Template(template_string)

In [59]:
prompt = template.render(data)

In [61]:
print(prompt)

I need to summarize a conversation. The transcript of the 
conversation is between the <data> XML like tags.

<data>

spk_0: Hi, is this the Crystal Heights Hotel in Singapore? 
spk_1: Yes, it is. Good afternoon. How may I assist you today? 
spk_0: Fantastic. Good afternoon. I was looking to book a room for my 10th wedding anniversary. I've heard your hotel offers exceptional views and services. Could you tell me more? 
spk_1: Absolutely, Alex, and congratulations on your upcoming anniversary. That's a significant milestone, and we'd be honored to make it a special occasion for you. We have several room types that offer stunning views of the city skyline and the fictional Sapphire Bay. Our special diamond suite even comes with exclusive access to the moonlit pool and star deck. We also have in-house spa services, world-class dining options, and a shopping arcade. 
spk_0: That sounds heavenly. I think my spouse would love the moonlit pool. Can you help me make a reservation for one of y

In [65]:
kwargs = {
    "modelId": "amazon.titan-text-express-v1",
    "contentType": "application/json",
    "accept": "*/*",
    "body": json.dumps(
        {
            "inputText": prompt,
            "textGenerationConfig": {
                "maxTokenCount": 512,
                "temperature": 0,
                "topP": 0.9
            }
        }
    )
}

In [67]:
response = bedrock_runtime.invoke_model(**kwargs)

In [69]:

response_body = json.loads(response.get('body').read())
generation = response_body['results'][0]['outputText']

In [71]:
print(generation)


{
    "sentiment": "neutral",
    "issues": [
        {
            "topic": "Price",
            "summary": "The guest is concerned about the $1000 pre-authorization hold on their credit card, which they consider excessive."
        },
        {
            "topic": "Additional charges",
            "summary": "The guest is informed about the 10% service charge and 7% fantasy tax applied to the room rate."
        }
    ]
}
